In [2]:
import tensorflow as tf
import numpy as np
from functools import partial
import pickle
import hashlib
import time, math
from loguru import logger
import configparser
import logging
import sys
import os
# import callbacks
from tensorflow.keras.callbacks import LearningRateScheduler, EarlyStopping
tf.get_logger().setLevel(logging.ERROR)

In [3]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

## Read the parameters specified in the configuration file

In [4]:
config = configparser.ConfigParser()
config.read('./modelconfig.ini')

['./modelconfig.ini']

In [5]:
tf.random.set_seed(int(config['model']['random_seed']))
np.random.seed(int(config['model']['random_seed']))

## Economic Model

In [6]:
class EconModel:
    def __init__(self, **kwargs):
        self.alpha = None
        self.beta = None
        self.rho = None
        self.sigma = None
        self.delta = None
        self.theta = None
        self.stdev = None
        self.k_lb = None
        
        for key, value in kwargs.items():
            if key in self.__dict__:
                self.__dict__[key] = value
            else:
                raise ValueError(f"Unknown model parameter {k} provided.\
                Valid parameters are: {[key for key in self.__dict__]}")
        
    @property
    def k_ub(self):
        return 10*self.k_nsss
    
    @property
    def k_nsss(self):
        return (self.alpha/(1/self.beta - (1-self.delta)))**(1/(1-self.alpha))
    
    @property
    def R(self):
        return self.alpha/(1.0 - self.beta*self.alpha)
    
    @property
    def S(self):
        return (1.0 + self.beta * self.R)/(1 - self.beta*self.rho)
    
    @property
    def Q(self):
        return (np.log(self.alpha /self.R) + self.beta*self.R*np.log(1 - self.alpha / self.R))/(1 - self.beta)
        
    def utility(self, c):
        if isinstance(c, tf.Tensor):
            return tf.math.log(c) if self.theta == 1 else (c ** (1.0 - self.theta) - 1.0)/(1.0 - self.theta)
        return np.log(c) if self.theta == 1 else (c ** (1.0 - self.theta) - 1.0)/(1.0 - self.theta)
        
    def marginal_utility(self, c):
        return c ** (-self.theta)
    
    def inv_marginal_utility(self, c):
        return c ** (-1.0/self.theta)
    
    def resources(self, k, z):
        return z * (k ** self.alpha) + (1.0 - self.delta)*k
    
    def dresources(self, k, z):
        return z * self.alpha * k ** (self.alpha - 1) + 1 - self.delta

## Custom loss function

In [7]:
def loss(V_true, V_predicted, dVdk, dVdz, kp, utility, beta):
        scale = 16.0
        if (dVdk.numpy() <= 0.0).any():
            return scale * tf.math.maximum(-dVdk, tf.zeros_like(dVdk))
        if (dVdz.numpy() <= 0.0).any():
            return scale * tf.math.maximum(-dVdz, tf.zeros_like(dVdz))
        if (kp.numpy() < 0.0).any():
            return scale * tf.math.maximum(-kp, tf.zeros_like(kp))
    
        loss = tf.math.abs(V_true - (utility+beta*V_predicted))
        return tf.math.reduce_mean(loss)

### Model training parameters

In [8]:
class EconModelDataGenerator(tf.keras.utils.Sequence):
    def __init__(self, econ_model: EconModel, dataset_size: int, batch_size: int):
        self.__model = econ_model
        self.dataset_size = dataset_size
        self.batch_size = batch_size
        
        # The static part of the dataset (Changes after all epochs)
        self.ks = tf.random.uniform(shape=[dataset_size], minval = self.__model.k_lb, maxval = self.__model.k_ub)
    
    def __len__(self):
        """
        Number of batches per epoch
        """
        return int(self.dataset_size/self.batch_size)
    
    def __getitem__(self, batch_index: int):
        """
        Yield a single batch of size `batch_size` from the dataset
        """
        # State sampler
        ks = self.ks[batch_index*self.batch_size:(batch_index+1)*self.batch_size]
        zs = tf.math.exp(self.__model.sigma*tf.random.normal(shape=[self.batch_size],
                                                             dtype = tf.dtypes.float32,
                                                             stddev = self.__model.stdev)/(1.0-self.__model.rho**2)**0.5)
        # Sample innovations
        zp1s = tf.math.exp(self.__model.rho*tf.math.log(zs) +
                           self.__model.sigma*tf.random.normal(shape=[self.batch_size],
                                                               stddev= self.__model.stdev))
        zp2s = tf.math.exp(self.__model.rho*tf.math.log(zs) +
                           self.__model.sigma*tf.random.normal(shape=[self.batch_size],
                                                               stddev=self.__model.stdev))
        
        zp1 = tf.convert_to_tensor(zp1s.numpy().reshape(-1, 1), dtype = tf.dtypes.float32)
        zp2 = tf.convert_to_tensor(zp2s.numpy().reshape(-1, 1), dtype = tf.dtypes.float32)
        k = tf.convert_to_tensor(ks.numpy().reshape(-1, 1), dtype = tf.dtypes.float32)
        z = tf.convert_to_tensor(zs.numpy().reshape(-1, 1), dtype = tf.dtypes.float32)
        return ((k, z, zp1, zp2),)

In [9]:
class ValueFunction(tf.keras.Model):
    def __init__(self, eco_model: EconModel, input_layer_dim: int):
        super(ValueFunction, self).__init__()
        self.__ecomodel = eco_model
        self.input_layer_dim = input_layer_dim
        
        initializer = tf.keras.initializers.RandomUniform(minval=0.01, maxval=0.125)
        self.inp = tf.keras.layers.InputLayer(input_shape=(self.input_layer_dim, 2))
        self.inp2 = tf.keras.layers.Flatten()
        self.fc1 = tf.keras.layers.Dense(64, activation = 'selu', bias_initializer='he_uniform')
        self.fc2 = tf.keras.layers.Dense(32,  activation = 'selu', bias_initializer='he_uniform')
        self.fc3 = tf.keras.layers.Dense(16,  activation = 'selu', bias_initializer='he_uniform')
        self.fc4 = tf.keras.layers.Dense(32,  activation = 'selu', bias_initializer='he_uniform') 
        self.fc5 = tf.keras.layers.Dense(32,   activation = 'selu', bias_initializer='he_uniform')
        self.fc6 = tf.keras.layers.Dense(2,   activation = 'selu', bias_initializer='he_uniform') 
        self.out = tf.keras.layers.Dense(1,   activation = 'linear', bias_initializer='he_uniform')
        #self.do1 = tf.keras.layers.Dropout(0.25)
        
    # Set forward pass.
    def call(self, input_data, training=False):
        x = input_data[0]
        z = input_data[1]
        x = tf.math.log(x)
        z = tf.math.log(z)
        y = tf.concat([x, z], axis = 1)
        #y = self.inp(y)
        #y = self.fc1(y)
        #y = self.fc2(y)
        #y = self.fc3(y)
        y = self.inp2(y)
        y = self.fc4(y)
        y = self.fc5(y)
        #y = self.fc6(y)
        y = self.out(y)
        return y
    
    # Custom training
    def train_step(self, data):
        k, z, zp1, zp2 = data[0]
        
        k = tf.Variable(k)
        z = tf.Variable(z)
        zp1 = tf.Variable(zp1)
        zp2 = tf.Variable(zp2)
        
        with tf.GradientTape(persistent=True) as tape:
            V = self((k, z,), training=True) # Forward pass
            
            # Gradients of the value function
            dVdk = tape.gradient(V, k)
            dVdz = tape.gradient(V, z)

            # Compute consumption
            c = self.__ecomodel.inv_marginal_utility(dVdk / self.__ecomodel.dresources(k, z))
            
            # Compute savings
            kp = self.__ecomodel.resources(k, z) - c
            
            Vp = self((kp, zp1), training=True)
            loss_avg = loss(V, Vp, dVdk, dVdz, kp, self.__ecomodel.utility(c), self.__ecomodel.beta)
            
        # Update weights
        #self.optimizer.minimize(loss_avg, self.trainable_variables, tape=tape)
        grads = tape.gradient(loss_avg, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_variables))
        self.compiled_metrics.update_state(V, self.__ecomodel.utility(c)+self.__ecomodel.beta*Vp)
        
        return {'loss': np.mean(loss_avg), 'consumption': tf.math.reduce_mean(c).numpy(), 'savings': tf.math.reduce_mean(kp).numpy()}

## Custom callbacks

In [10]:
class CustomCallbacks(tf.keras.callbacks.Callback):
    def __init__(self, epoch_step: int, results_filename: str, model_hash: str):
        self.epoch_step = epoch_step
        self.results_file = results_filename
        self.model_hash = model_hash
        self.__lrate = None
        self.__training_start_time = None
        self.__training_end_time = None
        self.__results_filehandle = None
        self.__loss = []
        self.__consumption = []
        self.__savings = []
        self.__results = {}
        
    def on_train_begin(self, logs=None):
        """
        Open pickle files to saves results to
        """
        self.__training_start_time = time.time()
        logger.info(f"Opening {self.results_file} to cache model training results...")
        self.__results["model"] = {}
        self.__results["model"]["id"] = self.model_hash
        self.__results["model"]["results"] = []
        self.__results_filehandle = open(self.results_file, 'ab')
        
    def on_epoch_end(self, epoch, logs):
        """
        Save losses, consumption and savings to pickle
        """
        self.__loss.append(logs['loss'])
        self.__consumption.append(logs['consumption'])
        self.__savings.append(logs['savings'])
        if epoch % self.epoch_step == 0:
            entry = {}
            entry["epoch"] = epoch
            entry["loss"] = np.mean(self.__loss)
            entry["consumption"] = np.mean(self.__consumption)
            entry["savings"] = np.mean(self.__savings)
            logger.info(f"Result: {entry}, lrate: {self.model.optimizer.lr.numpy()}")
            self.__results["model"]["results"].append(entry)
            
        self.__loss = []
        self.__consumption = []
        self.__savings = []
        
    def on_train_end(self, logs=None):
        """
        Cleanup after model training is done
        """
        self.__training_end_time = time.time()
        logger.info(f"Training finished in {self.__training_end_time-self.__training_start_time}")
        logger.info(f"Savings results to {self.results_file}")
        pickle.dump(self.__results, self.__results_filehandle)
        self.__results_filehandle.close()

## Learning rate scheduler

In [11]:
class MyLrScheduler(object):
    def __init__(self, epoch_steps: int, initial_rate: float):
        self.__epoch_steps = epoch_steps
        self.__lrate = initial_rate
        
    def __call__(self, epoch):
        if epoch % self.__epoch_steps == 0 and epoch > 0:
            self.__lrate /= 1.05 #math.pow(self.__lrate, math.floor((1+epoch)/self.__epoch_steps))
            #logger.info(f"Learning rate dropped to: {self.__lrate}")
        return self.__lrate

In [12]:
batch_size = int(config['model']['batch_size'])
econ_model_params = {"alpha": 0.36, "beta": 0.95,
                     "rho": 0.95, "sigma": 0.0,
                     "delta": 1.0, "theta": 1.0,
                     "k_lb": 1e-2, "stdev": 0.001/(1.0 - 0.2**2)**0.5}
em = EconModel(**econ_model_params)

In [13]:
vf = ValueFunction(em, input_layer_dim = batch_size)
lrate = float(config['model']['initial_learning_rate'])
vf.compile(optimizer=tf.keras.optimizers.Adam(lrate), run_eagerly=True)
model_hash = hashlib.sha1(str(hash(vf)).encode('utf-8')).hexdigest()[:10]

# results filename
timestr = time.strftime("%Y%m%d-%H%M%S")
results_filename = config['model']['results_filename']+f"_{timestr}.pkl"

# logging to file
log_file = f"{config['model']['log_file']}"+"_{time}.log"
fmt = "{message}"
logger.add(log_file, format=fmt)
logger.add(sys.stderr, format=fmt)

2

In [14]:
num_epochs = int(config['model']['num_epochs'])
epoch_step = int(config['model']['epoch_step'])

# fit
vf.fit(EconModelDataGenerator(em, batch_size, batch_size),
       epochs=num_epochs,
       callbacks=[
           CustomCallbacks(epoch_step=epoch_step, results_filename=results_filename, model_hash=model_hash),
           LearningRateScheduler(MyLrScheduler(int(config['model']['drop_lr_after']), lrate))
       #], verbose=0, workers=int(os.environ['SLURM_NTASKS']))
           ], verbose=0)

2021-01-20 08:01:31.946 | INFO     | __main__:on_train_begin:20 - Opening res_20210120-080131.pkl to cache model training results...
Opening res_20210120-080131.pkl to cache model training results...
2021-01-20 08:01:32.047 | INFO     | __main__:on_epoch_end:39 - Result: {'epoch': 0, 'loss': 4.07481, 'consumption': 0.7317869, 'savings': 0.23070878}, lrate: 0.0010000000474974513
Result: {'epoch': 0, 'loss': 4.07481, 'consumption': 0.7317869, 'savings': 0.23070878}, lrate: 0.0010000000474974513
2021-01-20 08:01:36.243 | INFO     | __main__:on_epoch_end:39 - Result: {'epoch': 100, 'loss': 1.4150074, 'consumption': 0.30783176, 'savings': 0.6546639}, lrate: 0.0006139132310636342
Result: {'epoch': 100, 'loss': 1.4150074, 'consumption': 0.30783176, 'savings': 0.6546639}, lrate: 0.0006139132310636342
2021-01-20 08:01:40.429 | INFO     | __main__:on_epoch_end:39 - Result: {'epoch': 200, 'loss': 1.3923098, 'consumption': 0.31443155, 'savings': 0.64806414}, lrate: 0.0003768894821405411
Result: {'

2021-01-20 08:03:10.790 | INFO     | __main__:on_epoch_end:39 - Result: {'epoch': 2600, 'loss': 1.3350524, 'consumption': 0.33280993, 'savings': 0.62968576}, lrate: 3.0958666741298657e-09
Result: {'epoch': 2600, 'loss': 1.3350524, 'consumption': 0.33280993, 'savings': 0.62968576}, lrate: 3.0958666741298657e-09
2021-01-20 08:03:14.768 | INFO     | __main__:on_epoch_end:39 - Result: {'epoch': 2700, 'loss': 1.3350525, 'consumption': 0.33280993, 'savings': 0.62968576}, lrate: 1.9005934603910646e-09
Result: {'epoch': 2700, 'loss': 1.3350525, 'consumption': 0.33280993, 'savings': 0.62968576}, lrate: 1.9005934603910646e-09
2021-01-20 08:03:18.486 | INFO     | __main__:on_epoch_end:39 - Result: {'epoch': 2800, 'loss': 1.3350525, 'consumption': 0.33280993, 'savings': 0.62968576}, lrate: 1.1667995458708447e-09
Result: {'epoch': 2800, 'loss': 1.3350525, 'consumption': 0.33280993, 'savings': 0.62968576}, lrate: 1.1667995458708447e-09
2021-01-20 08:03:21.806 | INFO     | __main__:on_epoch_end:39 - 

Result: {'epoch': 5200, 'loss': 1.3350525, 'consumption': 0.33280993, 'savings': 0.62968576}, lrate: 9.584390403452169e-15
2021-01-20 08:04:52.935 | INFO     | __main__:on_epoch_end:39 - Result: {'epoch': 5300, 'loss': 1.3350525, 'consumption': 0.33280993, 'savings': 0.62968576}, lrate: 5.883984248320146e-15
Result: {'epoch': 5300, 'loss': 1.3350525, 'consumption': 0.33280993, 'savings': 0.62968576}, lrate: 5.883984248320146e-15
2021-01-20 08:04:56.745 | INFO     | __main__:on_epoch_end:39 - Result: {'epoch': 5400, 'loss': 1.3350525, 'consumption': 0.33280993, 'savings': 0.62968576}, lrate: 3.612255859827742e-15
Result: {'epoch': 5400, 'loss': 1.3350525, 'consumption': 0.33280993, 'savings': 0.62968576}, lrate: 3.612255859827742e-15
2021-01-20 08:04:59.952 | INFO     | __main__:on_epoch_end:39 - Result: {'epoch': 5500, 'loss': 1.3350525, 'consumption': 0.33280993, 'savings': 0.62968576}, lrate: 2.2176116652505634e-15
Result: {'epoch': 5500, 'loss': 1.3350525, 'consumption': 0.33280993,

Result: {'epoch': 7900, 'loss': 1.3350525, 'consumption': 0.33280993, 'savings': 0.62968576}, lrate: 1.8216029277974993e-20
2021-01-20 08:06:26.524 | INFO     | __main__:on_epoch_end:39 - Result: {'epoch': 8000, 'loss': 1.3350525, 'consumption': 0.33280993, 'savings': 0.62968576}, lrate: 1.1183062636855689e-20
Result: {'epoch': 8000, 'loss': 1.3350525, 'consumption': 0.33280993, 'savings': 0.62968576}, lrate: 1.1183062636855689e-20
2021-01-20 08:06:30.019 | INFO     | __main__:on_epoch_end:39 - Result: {'epoch': 8100, 'loss': 1.3350525, 'consumption': 0.33280993, 'savings': 0.62968576}, lrate: 6.865430255334638e-21
Result: {'epoch': 8100, 'loss': 1.3350525, 'consumption': 0.33280993, 'savings': 0.62968576}, lrate: 6.865430255334638e-21
2021-01-20 08:06:33.538 | INFO     | __main__:on_epoch_end:39 - Result: {'epoch': 8200, 'loss': 1.3350525, 'consumption': 0.33280993, 'savings': 0.62968576}, lrate: 4.2147784500224775e-21
Result: {'epoch': 8200, 'loss': 1.3350525, 'consumption': 0.332809

In [15]:
vf.save('vf_keras_model')

C:\Program files\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:2291: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '
C:\Program files\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\base_layer.py:1377: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`layer.updates` will be removed in a future version. '
